<a href="https://colab.research.google.com/github/jarekwan/jarwan_projekt/blob/main/Projekt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BIBLIOTEKI I POBRANIE JSON Z RESTCOUNTRIES

In [14]:
import requests      # biblioteka do pobierania danych z API
import pandas as pd  # biblioteka do pracy z tabelami (DataFrame)
import json
from typing import List, Dict, Optional
from pydantic import BaseModel, ValidationError  # Pydantic do walidacji danych
# adres API RestCountries z wybranymi polami (max 10 na raz)
url = "https://restcountries.com/v3.1/all?fields=name,capital,region,subregion,population,area,currencies,languages,timezones,flags"

# pobranie danych z API
resp = requests.get(url)


#sprawdzanie surowego jsona
# wypisanie statusu odpowiedzi (200 = OK)
print("Status:", resp.status_code)

print(len(resp.text))#po nim wstawiam PYDANTIC
print(resp.text[:500])

# czy zaweira kluczowe frazy
print("name" in resp.text)
print("population" in resp.text)


Status: 200
170660
[{"flags":{"png":"https://flagcdn.com/w320/jm.png","svg":"https://flagcdn.com/jm.svg","alt":"The flag of Jamaica is divided by a gold diagonal cross into four alternating triangular areas of green at the top and bottom, and black on the hoist and fly sides"},"name":{"common":"Jamaica","official":"Jamaica","nativeName":{"eng":{"official":"Jamaica","common":"Jamaica"},"jam":{"official":"Jamaica","common":"Jamaica"}}},"currencies":{"JMD":{"name":"Jamaican dollar","symbol":"$"}},"capital":["Kingston
True
True


SPRAWDZANIE CZY JSON POBRAL SIE PRAWIDLOWO

In [15]:
if resp.status_code == 200:
    # konwersja odpowiedzi JSON → Python (lista słowników)
    data = resp.json()
    print(type(data))
    # sprawdź typ pierwszego elementu
    print(type(data[0]))
    # zamiana listy słowników na tabelę Pandas



else:
    # jeśli status ≠ 200, wyświetlamy komunikat błędu
    print("Błąd:", resp.text)

<class 'list'>
<class 'dict'>


SPRAWDZENIE 1SZY ELEMNTOW POSZCZEGOLNYCH POL JAKIEGO SA TYPU

In [8]:
print(type(data[0]["name"]))
print(type(data[0]["capital"]))
print(type(data[0]["region"]))
print(type(data[0]["subregion"]))
print(type(data[0]["population"]))
print(type(data[0]["area"]))
print(type(data[0]["currencies"]))
print(type(data[0]["languages"]))
print(type(data[0]["timezones"]))
print(type(data[0]["flags"]))


<class 'dict'>
<class 'list'>
<class 'str'>
<class 'str'>
<class 'int'>
<class 'float'>
<class 'dict'>
<class 'dict'>
<class 'list'>
<class 'dict'>


SPRAWDZENIE WEWNATRZ WSZYSTKICH ZAGNIEZDZEN ILE ICH JEST I JAKIEGO SA TYPU ZA POMACA REKURENCYJNEJ FUNKCJI

In [18]:
def describe(val):
    # jeśli to słownik
    if isinstance(val, dict):
        return {k: describe(v) for k, v in val.items()}
    # jeśli to lista
    elif isinstance(val, list) and len(val) > 0:
        return {"list_of": describe(val[0])}
    # proste typy
    else:
        return type(val).__name__

# użycie:
sample = data[0]  # pierwszy rekord z JSON
field_types = {k: describe(v) for k, v in sample.items()}

print(field_types)

{'flags': {'png': 'str', 'svg': 'str', 'alt': 'str'}, 'name': {'common': 'str', 'official': 'str', 'nativeName': {'eng': {'official': 'str', 'common': 'str'}, 'jam': {'official': 'str', 'common': 'str'}}}, 'currencies': {'JMD': {'name': 'str', 'symbol': 'str'}}, 'capital': {'list_of': 'str'}, 'region': 'str', 'subregion': 'str', 'languages': {'eng': 'str', 'jam': 'str'}, 'area': 'float', 'population': 'int', 'timezones': {'list_of': 'str'}}


PRZEDSTAWIENIE STRURY I TYPY WEW. JSON PLIKU W SPOSOB UMOZLIWJACA NAOCZNA WERYFIKACJE

In [21]:


print(json.dumps(field_types, indent=2))


{
  "flags": {
    "png": "str",
    "svg": "str",
    "alt": "str"
  },
  "name": {
    "common": "str",
    "official": "str",
    "nativeName": {
      "eng": {
        "official": "str",
        "common": "str"
      },
      "jam": {
        "official": "str",
        "common": "str"
      }
    }
  },
  "currencies": {
    "JMD": {
      "name": "str",
      "symbol": "str"
    }
  },
  "capital": {
    "list_of": "str"
  },
  "region": "str",
  "subregion": "str",
  "languages": {
    "eng": "str",
    "jam": "str"
  },
  "area": "float",
  "population": "int",
  "timezones": {
    "list_of": "str"
  }
}


STARE KODY KTORYCH NIE UZYWAM

In [9]:
# sample = data[0]  # pierwszy rekord
# field_types = {key: type(val).__name__ for key, val in sample.items()}
# print(field_types)

{'flags': 'dict', 'name': 'dict', 'currencies': 'dict', 'capital': 'list', 'region': 'str', 'subregion': 'str', 'languages': 'dict', 'area': 'float', 'population': 'int', 'timezones': 'list'}


In [11]:
# sample = data[0]  # pierwszy rekord

# field_types = {}
# for key, val in sample.items():
#     if isinstance(val, dict):
#         # słownik → sprawdzamy typy wartości wewnątrz
#         inner_types = {k: type(v).__name__ for k, v in val.items()}
#         field_types[key] = {"type": "dict", "inside": inner_types}
#     elif isinstance(val, list) and len(val) > 0:
#         # lista → sprawdzamy typ pierwszego elementu
#         field_types[key] = {"type": "list", "inside": type(val[0]).__name__}
#     else:
#         # proste typy (str, int, float) albo pusta lista
#         field_types[key] = {"type": type(val).__name__}

# print(field_types)

{'flags': {'type': 'dict', 'inside': {'png': 'str', 'svg': 'str', 'alt': 'str'}}, 'name': {'type': 'dict', 'inside': {'common': 'str', 'official': 'str', 'nativeName': 'dict'}}, 'currencies': {'type': 'dict', 'inside': {'JMD': 'dict'}}, 'capital': {'type': 'list', 'inside': 'str'}, 'region': {'type': 'str'}, 'subregion': {'type': 'str'}, 'languages': {'type': 'dict', 'inside': {'eng': 'str', 'jam': 'str'}}, 'area': {'type': 'float'}, 'population': {'type': 'int'}, 'timezones': {'type': 'list', 'inside': 'str'}}


PYDATNIC UZYWAM NA JSON , SPRAWDZAM ELEMENTY LISTY SLOWNIKOW JAKIM JEST JSON CZY SA ODPOWIEDNIEGO TYPU

PONIZEJ 1SZA NIEUDANA PROBA

In [22]:
# from typing import List, Dict, Optional
# from pydantic import BaseModel, ValidationError

# # definiujemy model danych dla pojedynczego kraju
# class Country(BaseModel):
#     name: Dict[str, str]                # nazwa kraju (dict, np. {common: Poland})
#     capital: Optional[List[str]]        # stolica (lista albo None)
#     region: Optional[str]               # region (np. Europe)
#     subregion: Optional[str]            # subregion (np. Eastern Europe)
#     population: int                     # populacja (int)
#     area: float                         # powierzchnia (float/int)
#     currencies: Optional[Dict]          # waluty (dict albo None)
#     languages: Optional[Dict]           # języki (dict albo None)
#     timezones: Optional[List[str]]      # strefy czasowe (lista stringów)
#     flags: Optional[Dict]               # flagi (dict np. png, svg)

# # walidacja – sprawdzamy każdy rekord z JSON
# valid = []
# invalid = []

# for kraj in data:
#     try:
#         c = Country(**kraj)         # walidacja pól
#         valid.append(c.dict())      # poprawne rekordy
#     except ValidationError as e:
#         invalid.append(kraj)        # błędne rekordy
#         print("❌ Błąd walidacji:", e)

# print(f"✔ Poprawnych rekordów: {len(valid)}")
# print(f"❌ Błędnych rekordów: {len(invalid)}")

# # dalej DataFrame tworzysz już z przefiltrowanych danych:
# df = pd.json_normalize(valid)


In [26]:
from typing import Dict, List
from pydantic import BaseModel

# -------- FLAGS --------
class Flags(BaseModel):             # odpowiada polu "flags"
    png: str
    svg: str
    alt: str

# -------- NAME --------
class Eng(BaseModel):               # dokładnie odpowiada "eng"
    official: str
    common: str

class Jam(BaseModel):               # dokładnie odpowiada "jam"
    official: str
    common: str

class NativeName(BaseModel):        # odpowiada "nativeName"
    eng: Eng
    jam: Jam

class Name(BaseModel):              # pole "name"
    common: str
    official: str
    nativeName: NativeName

# -------- CURRENCIES --------
class JMD(BaseModel):               # dokładnie odpowiada "JMD"
    name: str
    symbol: str

class Currencies(BaseModel):        # odpowiada "currencies"
    JMD: JMD

# -------- COUNTRY --------
class Country(BaseModel):           # cały rekord kraju
    flags: Flags
    name: Name
    currencies: Currencies
    capital: List[str]
    region: str
    subregion: str
    languages: Dict[str, str]
    area: float
    population: int
    timezones: List[str]




In [24]:
# from typing import Dict, List, Optional
# from pydantic import BaseModel

# # --- Najbardziej zagnieżdżone fragmenty ---

# class Flag(BaseModel):                   # pole "flags"
#     png: str
#     svg: str
#     alt: Optional[str]

# class NativeNameDetail(BaseModel):       # elementy wewnątrz "nativeName"
#     official: str
#     common: str

# class Name(BaseModel):                   # pole "name"
#     common: str
#     official: str
#     nativeName: Dict[str, NativeNameDetail]

# class CurrencyDetail(BaseModel):         # elementy wewnątrz "currencies"
#     name: str
#     symbol: str

# # --- Główna klasa kraju ---
# class Country(BaseModel):
#     flags: Flag
#     name: Name
#     currencies: Dict[str, CurrencyDetail]
#     capital: Optional[List[str]]
#     region: str
#     subregion: str
#     languages: Dict[str, str]
#     area: float
#     population: int
#     timezones: List[str]


In [4]:


    # TU WSTAWIAM PYDANTIC
    df = pd.json_normalize(data)
    # liczba krajów w danych
    print("Liczba krajów:", len(df))
    # podgląd pierwszych 5 wierszy
    print(df.head())


# df = pd.json_normalize(resp.json()) if resp.status_code == 200 else None
# print(df.head() if df is not None else resp.text)

Liczba krajów: 250
         capital    region           subregion      area  population  \
0     [Kingston]  Americas           Caribbean   10991.0     2961161   
1       [Moroni]    Africa      Eastern Africa    1862.0      869595   
2       [Harare]    Africa     Southern Africa  390757.0    14862927   
3  [Philipsburg]  Americas           Caribbean      34.0       40812   
4    [Vientiane]      Asia  South-Eastern Asia  236800.0     7275556   

     timezones                        flags.png                   flags.svg  \
0  [UTC-05:00]  https://flagcdn.com/w320/jm.png  https://flagcdn.com/jm.svg   
1  [UTC+03:00]  https://flagcdn.com/w320/km.png  https://flagcdn.com/km.svg   
2  [UTC+02:00]  https://flagcdn.com/w320/zw.png  https://flagcdn.com/zw.svg   
3  [UTC-04:00]  https://flagcdn.com/w320/sx.png  https://flagcdn.com/sx.svg   
4  [UTC+07:00]  https://flagcdn.com/w320/la.png  https://flagcdn.com/la.svg   

                                           flags.alt   name.common  ...  

In [ ]:
# sprawdzamy kilka pierwszych rekordów
for i, kraj in enumerate(data[:5]):
    try:
        # walidacja typów pól:
        assert isinstance(kraj.get("name"), dict)                  # nazwa kraju = słownik
        assert isinstance(kraj.get("capital"), list) or kraj.get("capital") is None  # stolica = lista lub None
        assert isinstance(kraj.get("population"), int)             # populacja = int
        assert isinstance(kraj.get("area"), (int, float))          # powierzchnia = int/float
        assert isinstance(kraj.get("currencies"), dict) or kraj.get("currencies") is None
        assert isinstance(kraj.get("languages"), dict) or kraj.get("languages") is None
        assert isinstance(kraj.get("region"), str) or kraj.get("region") is None      # region = string
        assert isinstance(kraj.get("subregion"), str) or kraj.get("subregion") is None # subregion = string
        assert isinstance(kraj.get("timezones"), list) or kraj.get("timezones") is None # strefy czasowe = lista
        assert isinstance(kraj.get("flags"), dict) or kraj.get("flags") is None        # flaga = dict

        print(f"✔ Rekord {i} poprawny")
    except AssertionError:
        print(f"❌ Błąd w rekordzie {i}: {kraj}")



✔ Rekord 0 poprawny
✔ Rekord 1 poprawny
✔ Rekord 2 poprawny
✔ Rekord 3 poprawny
✔ Rekord 4 poprawny
